In [14]:
from SPARQLWrapper import SPARQLWrapper, XML
import pandas as pd
import json

In [15]:
def get_sparql_dataframe(service, query):
    '''
    convert SPARQL results into a Pandas data frame.
    '''
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat('json')
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [17]:
service = 'http://10.20.241.27:7200/repositories/WEB2019'
# find all the restaurants surrounding the track (just show 20 of them)
query = '''
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

prefix : <http://www.topografix.com/GPX/1/1/> 
prefix osm: <https://www.openstreetmap.org/export/> 
prefix dbpedia: <http://dbpedia.org/ontology/> 
prefix wikidata: <https://www.wikidata.org/wiki/> 
prefix wikipedia: <https://en.wikipedia.org/wiki/> 

select ?trkname ?nd ?lat ?lon ?name ?place

where {?trk a :GPXtrack; :hasname ?trkname; :haswpt ?wpt; :hassurroundingnds ?nd. ?nd osm:amenity ?place; osm:hasname ?name; osm:haslat ?lat; osm:haslon ?lon. filter(?place = dbpedia:restaurant)} limit20
    
'''
get_sparql_dataframe(service, query)

,trkname,nd,lat,lon,name,place


In [18]:
# find all the amendities surrounding the track
query = '''
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

prefix : <http://www.topografix.com/GPX/1/1/> 
prefix osm: <https://www.openstreetmap.org/export/> 
prefix dbpedia: <http://dbpedia.org/ontology/> 
prefix wikidata: <https://www.wikidata.org/wiki/> 
prefix wikipedia: <https://en.wikipedia.org/wiki/> 

select ?n ?nd ?lat ?lon ?place ?wiki

where {?trk a :GPXtrack; :hasname ?n; :haswpt ?wpt; :hassurroundingnds ?nd. ?nd osm:amenity ?place; osm:haslat ?lat; osm:haslon ?lon; osm:wikidata ?wiki. filter(?place != dbpedia:none)}

'''
get_sparql_dataframe(service, query)

,n,nd,lat,lon,place,wiki


In [19]:
# find the tourisms surrounding the track
query = '''
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 

prefix : <http://www.topografix.com/GPX/1/1/> 
prefix osm: <https://www.openstreetmap.org/export/> 
prefix dbpedia: <http://dbpedia.org/ontology/> 
prefix wikidata: <https://www.wikidata.org/wiki/> 
prefix wikipedia: <https://en.wikipedia.org/wiki/> 

select ?trkname ?nd ?lat ?lon ?place

where {?trk a :GPXtrack; :hasname ?trakname; :haswpt ?wpt; :hassurroundingnds ?nd. ?nd osm:tourism ?place; osm:haslat ?lat; osm:haslon ?lon. filter(?place != dbpedia:none)} limit20
'''
get_sparql_dataframe(service, query)

,trkname,nd,lat,lon,place
